In [61]:
import h3
from bitarray import bitarray
from bitarray.util import ba2int, ba2hex
import toolz

def to_lower(h):
    d = h3.string_to_h3(h)
    b = '{0:064b}'.format(d)

    return -ba2int(bitarray(b)[-52:])

def rep(h):
    d = h3.string_to_h3(h)
    b = '{0:064b}'.format(d)[-52:]
    
    return b



def foo(h):
    d = h3.string_to_h3(h)
    b = '{0:064b}'.format(d)[-45:]
    
    g = toolz.partition(3, b)
    g = (''.join(e) for e in g)
    g = map(bitarray, g)
    g = map(ba2int, g)
    g = filter(lambda x: x != 7, g)
    g = list(g)
    
    return g
    
def descr(h):
    return h3.h3_get_base_cell(h), foo(h)

In [4]:
## mixed resolutions

h = h3.geo_to_h3(0,0,2)

p = list(h3.k_ring(h,1))[:3]
hexes = [
    h3.uncompact(p, 2),
    list(h3.uncompact(p, 3)),
    list(h3.uncompact(p, 4)),
]

hexes = set.union(*hexes)

In [2]:
## single resolution

h = h3.geo_to_h3(0,0,2)

hexes = h3.h3_to_children(h, 4)

In [9]:
seq = list(sorted(hexes, key=to_lower, reverse=True))

list(map(descr, seq))[:10]

[(58, [5, 1, 0, 0]),
 (58, [5, 1, 0, 1]),
 (58, [5, 1, 0, 2]),
 (58, [5, 1, 0, 3]),
 (58, [5, 1, 0, 4]),
 (58, [5, 1, 0, 5]),
 (58, [5, 1, 0, 6]),
 (58, [5, 1, 1, 0]),
 (58, [5, 1, 1, 1]),
 (58, [5, 1, 1, 2])]

In [11]:
h3.h3_to_parent(h)

'81757ffffffffff'

In [21]:
#seq

cur_parent = []
cur_children = []
working = []

def reset(e):
    cur_parent[:] = [p]
    cur_children[:] = [1]
    working[:] = [e]

def acc(e):
    p = h3.h3_to_parent(e)
    
    if cur_parent:
        if cur_parent[-1] == p:
            working.append(e)
            cur_children[-1] = cur_children[-1] + 1
            # what if it is 7?
            # blahhhh
        else:
            for w in working:
                yield w
            reset(e)
            
        if cur_children[-1] == 7:
            p = cur_parent.pop()
            cur_children.pop()
            for _ in range(7):
                working.pop()
            
            
            # pop in p
        
    if not cur_parent:
        reset(e)
        

In [13]:
parent = [1,2]

def blah():
    parent[:] = []

In [14]:
parent

[1, 2]

In [15]:
blah()

In [19]:
parent.pop?

Signature: parent.pop(index=-1, /)
Docstring:
Remove and return item at index (default last).

Raises IndexError if list is empty or index is out of range.
Type:      builtin_function_or_method


In [22]:
toolz.peek?

Signature: toolz.peek(seq)
Docstring:
Retrieve the next element of a sequence

Returns the first element and an iterable equivalent to the original
sequence, still having the element retrieved.

>>> seq = [0, 1, 2, 3, 4]
>>> first, seq = peek(seq)
>>> first
0
>>> list(seq)
[0, 1, 2, 3, 4]
File:      ~/work/marketbalance/env/lib/python3.7/site-packages/toolz/itertoolz.py
Type:      function


In [82]:
# don't need the working set. we can work in-place!


def is_child(p, c):
    r = h3.h3_get_resolution(p)
    return p == h3.h3_to_parent(c, r)

def do_it(seq):
    cur_parent = []
    cur_children = []
    working = []

    seq = list(reversed(list(seq)))
    
    while seq:
        e = seq.pop()
        p = h3.h3_to_parent(e)
            
        if not cur_parent:
            cur_parent = [p]
            cur_children = [1]
            working = [e]
            continue

        if cur_parent[-1] == p:
            working.append(e)
            cur_children[-1] += 1
            if cur_children[-1] == 7:
                p = cur_parent.pop()
                cur_children.pop()
                for _ in range(7):
                    working.pop()
                seq.append(p)
        elif is_child(cur_parent[-1], e):
            cur_parent.append(p)
            cur_children.append(1)
            working.append(e)
        else:
            for w in working:
                yield w
            cur_parent = []
            cur_children = []
            working = []
            seq.append(e)
            
    for w in working:
        yield w

In [83]:
seq = list(sorted(hexes, key=to_lower, reverse=True))

In [84]:
out = list(do_it(seq))
out

['82754ffffffffff']